# 数据聚合与分组运算
对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），通常是数据分析工作中的重要环节。在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成透视表。pandas提供了一个灵活高效的gruopby功能，它使你能以一种自然的方式对数据集进行切片、切块、摘要等操作。

关系型数据库和SQL（Structured Query Language，结构化查询语言）能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。但是，像SQL这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到，由于Python和pandas强大的表达能力，我们可以执行复杂得多的分组运算（利用任何可以接受pandas对象或NumPy数组的函数）。在本章中，你将会学到：

* 使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象。
* 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
* 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
* 计算透视表或交叉表。
* 执行分位数分析以及其它统计分组分析。
## Group by机制
![how-to-groupby](images/groupby.png)

In [34]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],'key2' : ['one', 'two', 'one', 'two', 'one'],'data1' : np.random.randn(5),'data2' : np.random.randn(5)})
print(df)
# print(df.index)
# summmury = df['data1'].groupby(df['key1']).sum()#group by的写法
# print(summmury)
# summmury2 = df['data1'].groupby([df['key1'],df['key2']]).sum()#group by可以多个字段，注意group by([])
# print(summmury2)
# print(summmury2.unstack())#层次化索引,看的更清晰一些
# states = np.array(['Ohio', 'Ohio', 'California', 'California', 'Ohio'])
# years = np.array([2005, 2006, 2005, 2006, 2005])
#打印数据
print(df['data1'])
print(df['data1'].groupby(df['key1']).sum())
# print(states)
# print(years)
# print(df['data1'].groupby([states, years]).sum())#其中states就是key1，years就是key2，只是其中的内容变成了states和years，类似sql中的groupby sum
# print(df.groupby(['key1', 'key2']).size())#类似sql中的groupby count




  key1 key2     data1     data2
0    a  one -0.001708 -0.578242
1    a  two  0.797194 -0.040421
2    b  one  1.269027  0.322051
3    b  two -0.519620 -0.760153
4    a  one -0.478164 -1.497868
0   -0.001708
1    0.797194
2    1.269027
3   -0.519620
4   -0.478164
Name: data1, dtype: float64


## 对分组进行迭代
GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。看下面的例子：

In [27]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],'key2' : ['one', 'two', 'one', 'two', 'one'],'data1' : np.random.randn(5),'data2' : np.random.randn(5)})
print(df)
for name, group in df.groupby('key1'):#迭代打印
   print(name)#打印key1分组中分组名称
   print(group)#打印分组信息

  key1 key2     data1     data2
0    a  one -0.595198 -0.581745
1    a  two -1.375262  0.054339
2    b  one  1.261285  0.603038
3    b  two -0.711359  1.013885
4    a  one  0.021047  1.038116
a
  key1 key2     data1     data2
0    a  one -0.595198 -0.581745
1    a  two -1.375262  0.054339
4    a  one  0.021047  1.038116
b
  key1 key2     data1     data2
2    b  one  1.261285  0.603038
3    b  two -0.711359  1.013885


对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

In [28]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
  print((k1, k2))
  print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.595198 -0.581745
4    a  one  0.021047  1.038116
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.375262  0.054339
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.261285  0.603038
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.711359  1.013885


当然，你可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：将这些数据片段做成一个字典：

In [33]:
pieces = dict(list(df.groupby('key1')))
pieces#还可以以key1为key

{'a':   key1 key2     data1     data2
 0    a  one -0.595198 -0.581745
 1    a  two -1.375262  0.054339
 4    a  one  0.021047  1.038116,
 'b':   key1 key2     data1     data2
 2    b  one  1.261285  0.603038
 3    b  two -0.711359  1.013885}